# Nissen et al., 2023: Severe 21st-century OA in Antarctic MPAs
#
# script to save top 200m averaged fields of omega aragonite, omega calcite and pH as netcdf files
# requires full model output
# note that some paths are hard-coded throughout the script

In [1]:
import sys
import os
import matplotlib.pyplot as plt
import numpy as np
from scipy.interpolate import griddata
from matplotlib import cm
from netCDF4 import Dataset, MFDataset
import pandas as pd
from tqdm import tqdm

In [2]:
#------
# MODEL INFO
#------
    
# load mesh
path_mesh = '/pscratch/sd/c/cnissen/'
file_mesh = 'Nissen2022_FESOM_REcoM_mesh_information_corrected_20220910.nc'

f1 = Dataset(path_mesh+file_mesh) #xr.open_dataset(path+file1)
lat      = f1.variables['lat'][:]
lon      = f1.variables['lon'][:]
zlevs    = f1.variables['zlevs'][:]
cavities = f1.variables['cavity'][:]
topo     = f1.variables['topo'][:]
area     = f1.variables['cell_area'][:]
volume   = f1.variables['cell_volume'][:]
f1.close()
print(lat.shape)
ind_cavities = np.where(cavities==1)[0]


(72411,)


In [36]:
#-----
# load model data & extract bottom layer omega A, omega C, pH 
#-----

which_sim = 'simAssp585'

path = '/pscratch/sd/c/cnissen/carbonate_chemistry_3D_v2/'+which_sim+'/' 

vari_list = ['omegaA_top200','omegaC_top200','pH_top200']

#year_list1 = np.arange(2090,2099+1,1) 
year_list1 = np.arange(1990,1999+1,1) 

# get depth levels
ind_dd = np.where((zlevs<=200))[0]
print(ind_dd)

omegaA_model1 = np.zeros([len(year_list1),len(lon)])
omegaC_model1 = np.zeros([len(year_list1),len(lon)])
pH_model1     = np.zeros([len(year_list1),len(lon)])
for yy in range(0,len(year_list1)):
    print('Process year '+str(year_list1[yy])+' of '+which_sim)
    
    path  = '/pscratch/sd/c/cnissen/carbonate_chemistry_3D_v2/'+which_sim+'/'
    f1    = Dataset(path+'OmegaA_fesom_'+str(year_list1[yy])+'0101.nc') 
    f2    = Dataset(path+'OmegaC_fesom_'+str(year_list1[yy])+'0101.nc') 
    f3    = Dataset(path+'pH_fesom_'+str(year_list1[yy])+'0101.nc')
    data_mean1 = f1.variables['OmegaA'][:,:,:].mean(axis=0)  # depth x mesh.n2d
    data_mean2 = f2.variables['OmegaC'][:,:,:].mean(axis=0) 
    data_mean3 = f3.variables['pH'][:,:,:].mean(axis=0) 
    f1.close()
    f2.close()
    f3.close()
    
    #----
    # get top 200m average
    #----
    omegaA_1 = np.zeros(len(lon))
    omegaC_1 = np.zeros(len(lon))
    pH_1     = np.zeros(len(lon))
    for nn in range(0,len(lon)):
        if cavities[nn]==0: # only outside of cavity
            vol = volume[nn,ind_dd]
            # correct vol for any missing data
            vol[data_mean1[ind_dd,nn].mask==True] = np.nan
            omegaA_1[nn] = np.nansum(data_mean1[ind_dd,nn]*vol/np.nansum(vol))
            omegaC_1[nn] = np.nansum(data_mean2[ind_dd,nn]*vol/np.nansum(vol))
            pH_1[nn]     = np.nansum(data_mean3[ind_dd,nn]*vol/np.nansum(vol))
    del data_mean1,data_mean2,data_mean3
    
    omegaA_model1[yy,:] = omegaA_1 
    omegaC_model1[yy,:] = omegaC_1 
    pH_model1[yy,:]     = pH_1 
    del pH_1,omegaA_1,omegaC_1
    
# get decadal avg
omegaA_model1 = np.nanmean(omegaA_model1,axis=0)
omegaC_model1 = np.nanmean(omegaC_model1,axis=0)
pH_model1     = np.nanmean(pH_model1,axis=0)

# mask unavailable nodes
omegaA_model1 = np.ma.masked_where(omegaA_model1==0,omegaA_model1)
omegaC_model1 = np.ma.masked_where(omegaC_model1==0,omegaC_model1)
pH_model1     = np.ma.masked_where(pH_model1==0,pH_model1)

print('Min/Max omegaA:',np.nanmin(omegaA_model1),np.nanmax(omegaA_model1))
print('Min/Max omegaC:',np.nanmin(omegaC_model1),np.nanmax(omegaC_model1))
print('Min/Max pH:',np.nanmin(pH_model1),np.nanmax(pH_model1))


[ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18]
Process year 1990 of simAssp585


/tmp/ipykernel_1532736/3085811957.py:47: UserWarning: Warning: converting a masked element to nan.
  omegaA_1[nn] = np.nansum(data_mean1[ind_dd,nn]*vol/np.nansum(vol))
/tmp/ipykernel_1532736/3085811957.py:48: UserWarning: Warning: converting a masked element to nan.
  omegaC_1[nn] = np.nansum(data_mean2[ind_dd,nn]*vol/np.nansum(vol))
/tmp/ipykernel_1532736/3085811957.py:49: UserWarning: Warning: converting a masked element to nan.
  pH_1[nn]     = np.nansum(data_mean3[ind_dd,nn]*vol/np.nansum(vol))


Process year 1991 of simAssp585
Process year 1992 of simAssp585
Process year 1993 of simAssp585
Process year 1994 of simAssp585
Process year 1995 of simAssp585
Process year 1996 of simAssp585
Process year 1997 of simAssp585
Process year 1998 of simAssp585
Process year 1999 of simAssp585
Min/Max omegaA: 1.0282437511352684 1.8669194999285925
Min/Max omegaC: 1.6385224358871777 2.9560018209377485
Min/Max pH: 7.910150632670879 8.149474409247029


/tmp/ipykernel_1532736/3085811957.py:58: RuntimeWarning: Mean of empty slice
  omegaA_model1 = np.nanmean(omegaA_model1,axis=0)
/tmp/ipykernel_1532736/3085811957.py:59: RuntimeWarning: Mean of empty slice
  omegaC_model1 = np.nanmean(omegaC_model1,axis=0)
/tmp/ipykernel_1532736/3085811957.py:60: RuntimeWarning: Mean of empty slice
  pH_model1     = np.nanmean(pH_model1,axis=0)


In [37]:
# -----
# save decadal avg as netcdf file
#-----
    
save_netcdf = True
# only save every year (takes too much time otherwise)
if save_netcdf:
    savepath = '/pscratch/sd/c/cnissen/carbonate_chemistry_3D_v2/top200m_fields/'
    
    if year_list1[0]==2090:
        time_str = '2090s'
    elif year_list1[0]==1990:
        time_str = '1990s'
    
    for vv in range(0,len(vari_list)):
        print(vari_list[vv])
        if year_list1[0]==1990:
            netcdf_name = vari_list[vv]+'_'+time_str+'_'+which_sim[:-6]+'.nc'
        else:
            netcdf_name = vari_list[vv]+'_'+time_str+'_'+which_sim+'.nc'
        if not os.path.exists(savepath+netcdf_name):
            print('Create file '+savepath+netcdf_name)
            w_nc_fid = Dataset(savepath+netcdf_name, 'w', format='NETCDF4_CLASSIC')
            w_nc_fid.mocsy_source = 'https://github.com/jamesorr/mocsy'
            w_nc_fid.mocsy_dir    = '/home/ollie/ncara/scripts/mocsy/mocsy'
            w_nc_fid.script    = '/global/homes/c/cnissen/scripts/plot_PAPER_carbonate_chemistry_save_top200m_fields.ipynb'
            # create dimension & variable
            w_nc_fid.createDimension('nodes_2d', len(topo)) 
            w_nc_var1 = w_nc_fid.createVariable(vari_list[vv], 'f4',('nodes_2d'))
            w_nc_fid.close()
            
        else: # file exists, but check if variables exist as well
            w_nc_fid = Dataset(savepath+netcdf_name, 'r+', format='NETCDF4_CLASSIC') 
            try:
                w_nc_var1 = w_nc_fid.createVariable(vari_list[vv], 'f4',('nodes_2d'))
            except: 
                print (vari_list[vv]+' exists in file')
            w_nc_fid.close()
                    
        w_nc_fid = Dataset(savepath+netcdf_name, 'r+', format='NETCDF4_CLASSIC') 
        if vari_list[vv] in ['omegaA_top200']:
            w_nc_fid.variables[vari_list[vv]][:] = omegaA_model1[:]
            print('written omegaA')
        elif vari_list[vv] in ['omegaC_top200']:
            w_nc_fid.variables[vari_list[vv]][:] = omegaC_model1[:]
            print('written omegaC')
        elif vari_list[vv] in ['pH_top200']:
            w_nc_fid.variables[vari_list[vv]][:] = pH_model1[:]
            print('written pH')
        w_nc_fid.close()  
        del netcdf_name
                
print ('done')


omegaA_top200
Create file /pscratch/sd/c/cnissen/carbonate_chemistry_3D_v2/top200m_fields/omegaA_top200_1990s_simA.nc
written omegaA
omegaC_top200
Create file /pscratch/sd/c/cnissen/carbonate_chemistry_3D_v2/top200m_fields/omegaC_top200_1990s_simA.nc
written omegaC
pH_top200
Create file /pscratch/sd/c/cnissen/carbonate_chemistry_3D_v2/top200m_fields/pH_top200_1990s_simA.nc
written pH
done
